In [1]:
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ExponentialLR
import torch.optim as optim
from torchvision import models
from torchsummary import summary

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    #random.seed(seed)
    torch.backends.cudnn.deterministic = True

def read_bci_data():
    S4b_train = np.load('S4b_train.npz')
    X11b_train = np.load('X11b_train.npz')
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    train_data = np.concatenate((S4b_train['signal'], X11b_train['signal']), axis=0)
    train_label = np.concatenate((S4b_train['label'], X11b_train['label']), axis=0)
    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)

    train_label = train_label - 1
    test_label = test_label -1
    train_data = np.transpose(np.expand_dims(train_data, axis=1), (0, 1, 3, 2))
    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))

    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

    print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

    return train_data, train_label, test_data, test_label

/DATA/vin/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class EEGNet(nn.Module):
    def __init__(self, activation='relu', dropout=0.25):
        super(EEGNet, self).__init__()
        #super().__init__()

        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU()
        else:
            self.activation = nn.ELU(alpha=1.0)

        self.conv2d = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False),
            #nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            nn.BatchNorm2d(16)
        )
        self.depthwiseConv2d = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False),
            #nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.BatchNorm2d(32),
            self.activation,
            nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0),
            nn.Dropout(p=dropout)
        )
        self.separableConv2d = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False),
            nn.BatchNorm2d(32),
            #nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            self.activation,
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0),
            nn.Dropout(p=dropout)
        )
        self.classify = nn.Sequential(
            #nn.Flatten(),
            nn.Linear(in_features=736, out_features=2, bias=True)
        )

    def forward(self, x):
        out = self.conv2d(x)
        out = self.depthwiseConv2d(out)
        out = self.separableConv2d(out)
        #out = self.classify(out)
        out = self.classify(out.flatten(start_dim=1))
        return out

In [3]:
class DeepConvNet(nn.Module):
    def __init__(self, activation='relu'):
        #super().__init__()
        super(DeepConvNet, self).__init__()
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU()
        else:
            self.activation = nn.ELU(alpha=1.0)

        self.layer0 = nn.Sequential(
            nn.Conv2d(1, 25, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.Conv2d(25, 25, kernel_size=(2, 1), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(25),
            self.activation,
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5))

        self.layer1 = nn.Sequential(
            nn.Conv2d(25, 50, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(50),
            self.activation,
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5))

        self.layer2 = nn.Sequential(
            nn.Conv2d(50, 100, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(100),
            self.activation,
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5))

        self.layer3 = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(200),
            self.activation,
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5))

        self.layer4 = nn.Sequential(
            #nn.Flatten(),
            nn.Linear(in_features=43*200, out_features=2, bias=True))

    def forward(self, x):
        out = self.layer0(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.layer4(out)
        out = self.layer4(out.flatten(start_dim=1))
        return out

In [4]:
def train(model, device, train_loader, optimizer, criterion, learning_rate, epoch, gamma, log_period, test_loader):
    
    opt = optimizer(model.parameters(), lr=learning_rate,weight_decay = 0)
    scheduler = ExponentialLR(opt, gamma=gamma)
    loss_list = []

    for i in range(epoch):
        loss_sum = 0
        model.train()
        for data,target in train_loader:
            data, target = data.to(device), target.to(device)
            #target = target.to(torch.int64)
            
            opt.zero_grad()
            out = model(data)
            loss = criterion(out,target)
            loss.backward()
            opt.step()
            
            loss_sum += loss.cpu().item()
            loss_list.append(loss)
            
            
            
        #scheduler.step()
        acc = test(model,device,criterion,test_loader)
        if((i+1) % log_period == 0):
            print(f"Train epoch: {i+1}   loss: {loss_sum/(log_period*len(train_loader.dataset))}   acc: {acc}")
best_model = None
def test(model, device, criterion, test_loader):
    model.eval()
    correct = 0
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            #target = target.to(torch.int64)
            out = model.forward(data)
            test_loss += criterion(out,target)
            out = torch.argmax(out, dim=1)
            correct += (out==target).sum().cpu().item()
    test_loss /= len(test_loader.dataset)
    if(correct / len(test_loader.dataset)>=0.87):
        best_model = model
    acc = 100. * correct / len(test_loader.dataset)
    print(f"Average acc = {acc}")
    return acc

In [5]:
def main():
    setup_seed(87)
    train_x,train_y,test_x,test_y = read_bci_data()
    
    use_cuda = torch.cuda.is_available()
    
    device = torch.device("cuda" if use_cuda else "cpu")
    batch_size = 32
    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': len(train_y)}
    if use_cuda:
        print("Use GPU for training...")
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)
    else:
        print("Use CPU to training...")
        

    train_dataset = TensorDataset(torch.tensor(train_x, dtype=torch.float32),torch.tensor(train_y, dtype=torch.int64))
    test_dataset = TensorDataset(torch.tensor(test_x, dtype=torch.float32), torch.tensor(test_y, dtype=torch.int64))
    train_loader = DataLoader(train_dataset, **train_kwargs)
    test_loader = DataLoader(test_dataset, **train_kwargs)
    
    model = EEGNet(activation='lrelu').to(device)
    #print(model)
   
    #model = DeepConvNet(activation='lrelu').to(device)
    train(model=model, 
          device=device, 
          train_loader=train_loader, 
          optimizer=optim.RAdam, 
          criterion=nn.CrossEntropyLoss(),
          learning_rate=1e-3, 
          epoch=300, 
          gamma=0.98, 
          log_period=50,
          test_loader=test_loader)
    if(best_model != None):
        print("878787878787878787878787878787878787878787878787878")
    else:
        print("Not found")

main()

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)
Use GPU for training...
Average acc = 58.24074074074074
Average acc = 63.425925925925924
Average acc = 70.0
Average acc = 70.37037037037037
Average acc = 71.01851851851852
Average acc = 71.20370370370371
Average acc = 72.22222222222223
Average acc = 72.4074074074074
Average acc = 73.05555555555556
Average acc = 73.42592592592592
Average acc = 73.14814814814815
Average acc = 75.74074074074075
Average acc = 75.74074074074075
Average acc = 76.66666666666667
Average acc = 77.4074074074074
Average acc = 76.01851851851852
Average acc = 77.87037037037037
Average acc = 79.53703703703704
Average acc = 79.25925925925925
Average acc = 77.12962962962963
Average acc = 79.53703703703704
Average acc = 78.98148148148148
Average acc = 80.0925925925926
Average acc = 81.48148148148148
Average acc = 79.9074074074074
Average acc = 80.83333333333333
Average acc = 79.9074074074074
Average acc = 81.66666666666667
Average acc = 79.9074074074074
Average acc = 

Average acc = 86.38888888888889
Train epoch: 250   loss: 4.0218722068756404e-05   acc: 86.38888888888889
Average acc = 87.12962962962963
Average acc = 87.4074074074074
Average acc = 87.96296296296296
Average acc = 88.24074074074075
Average acc = 88.24074074074075
Average acc = 87.5
Average acc = 87.77777777777777
Average acc = 86.75925925925925
Average acc = 87.5925925925926
Average acc = 85.83333333333333
Average acc = 86.85185185185185
Average acc = 86.75925925925925
Average acc = 86.38888888888889
Average acc = 86.48148148148148
Average acc = 86.57407407407408
Average acc = 85.83333333333333
Average acc = 87.12962962962963
Average acc = 87.96296296296296
Average acc = 87.31481481481481
Average acc = 86.94444444444444
Average acc = 87.03703703703704
Average acc = 87.03703703703704
Average acc = 86.66666666666667
Average acc = 87.5925925925926
Average acc = 87.4074074074074
Average acc = 87.77777777777777
Average acc = 86.94444444444444
Average acc = 85.83333333333333
Average acc = 86